# Similarity
Now that we can produce colour palettes for images, we can move on to the second task - asserting similarity between images based on the colours present within them.  
For the task of producing notebooks, researching previous literature alone was sufficient to produce satisfactory results. However, because of the difference between our domain and more typical image datasets, I think it's worth implementing a few different approaches in code to determine which is most appropriate/effective. We'll start with histograms.

### Image histograms
As we saw in the last notebook, images are composed of pixels, and the colour of those pixels can be represented by three numbers (alternatively seen as coordinates in colour space). We can stack the pixels in an image to produce a table of three columns or channels (traditionally R, G, and B, where each channel can take on a value between 0 and 255 denoting its intensity). 
A common technique to assert the 'makeup' of an image is to produce a _histogram_ of these channels, counting the number of times that each intensity appears in each channel. Our goal in this notebook is to determine an effective way of comparing these histograms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (20, 15)

import os
import itertools
import numpy as np
import pandas as pd
from PIL import Image
from scipy.spatial.distance import cosine
from umap import UMAP

from tqdm import tqdm_notebook as tqdm

Lets randomly load in a few images and get a visual sense of the distribution of colours/tones/styles across our chosen dataset.

In [ ]:
n_images = 1000
path_to_images = '../data/small_images/'

image_ids = np.random.choice(os.listdir(path_to_images), n_images, replace=False)
images = [Image.open(path_to_images + image_id) for image_id in tqdm(image_ids)]
images = [Image.fromarray(np.stack((image,)*3, -1))
          if len(np.array(image).shape) != 3 else image
          for image in images]

In [ ]:
resolution = 75
size = int(n_images ** 0.5)

height = int(resolution * size)
width = int(resolution * size)

big_image = np.empty((height, width, 3)).astype(np.uint8)
grid = np.array(list(itertools.product(range(size), range(size))))
sq_images = [image.resize((resolution, resolution)) for image in images]

for pos, image in zip(grid, sq_images):
    block_t, block_l = pos * resolution
    block_b, block_r = (pos + 1) * resolution
    
    big_image[block_t : block_b, block_l : block_r] = np.array(image)

Image.fromarray(big_image)

### Inspecting histograms
Let's produce a histogram and get a feel for the kind of data we're working with

In [ ]:
image = images[np.random.choice(len(images))]
hist = image.histogram()

image

this is what our raw histogram data looks like

In [ ]:
pd.Series(hist).plot();

and here it is split out into its three colour channels in the 

In [ ]:
r, g, b = np.split(np.array(hist), 3)

pd.DataFrame({'r': r, 'g': g, 'b': b}).plot(color=['#333399', '#339933', '#993333']);

# A naive start
Lets blunder straight in with the simplest approach possible. We'll produce a historgram for each image as one long 756-dimensional vector and then brute force the cosine similarity of each image with every other image.

In [ ]:
histograms = [image.histogram() for image in images]

In [ ]:
naive_similarity = pd.DataFrame(data=[[cosine(h_1, h_2)
                                       for h_1 in histograms] 
                                      for h_2 in tqdm(histograms)],
                                index=image_ids,
                                columns=image_ids)

we can display the similarity matrix as a heatmap, as follows:

In [ ]:
sns.heatmap(naive_similarity);

In [ ]:
image_dict = dict(zip(image_ids, images))

It's now pretty easy to use our similarity matrix to look up the most similar images to a given images. We select a random image:

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

and then grab the column in the matrix which holds similarity measures for our `query_id`. We sort the indexes according to their similarity to the query image, and grab the top 5 results (which are not our original query)

In [ ]:
most_similar_ids = naive_similarity[query_id].sort_values().index.values[1:6]

we can now grab the corresponding images and display them

In [ ]:
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

these results are super sketchy... They're not _bad_ exactly, but they're definitely not good. Playing with the results for a while, it's easy to spot where the approach succeeds and where the rough edges appear.

### The problem with histograms
If you've spent any amount of time exploring the collection, you know that _a lot_ of the images exist in a weird, sad, grubby, greyscale/sepia section of the colour space. In that space, the straight histogram comparison approach is perfectly sufficient (I was actually surprised by how good the results are for some of these query images).  
The method falls down when 'unusual' colours towards the ends of the 3D colour space start appearing, especially when they exist in a narrow band of intensities.  
Cosine similarity (and almost all other vector similarity metrics) assume that neighbouring elements in the vector represent distances along axes that are _orthogonal_, ie independent from one another. To cosine, the histogram bins for `red=222` and `red=223` are _entirely_ different colours, while we know that to our eyes they appear very similar. We want our system to know that neighbouring values along our colour axes are functionally the same, and should be scored as similar.

# Smoothed histograms
This problem might be solved by smoothing our histograms. By taking a moving average (with window size empirically chosen as `w=10`), we blur the distinction between `red=222` and `red=223` etc.  
Surprisingly, numpy doesn't have a built in moving average function defined, so we'll use the following functions to do the operation for us.

In [ ]:
def moving_average(arr, w):
    '''
    Returns a moving average over a given array
    
    Parameters
    ----------
    arr : numpy.array
        input array
    n : int
        window size

    Returns
    -------
    arr : numpy.array
        input array with moving average applied
    '''
    cumsum = np.cumsum(arr)
    return (cumsum[w:] - cumsum[:-w])[w - 1:] / w


def smooth_histogram(hist, w=10):
    '''
    applies a moving average to a image histogram, retaining separation between
    the 3 channels
    
    Parameters
    ----------
    hist : list
        flat input histogram of size=(768,)
    n : int
        window size

    Returns
    -------
    arr : numpy.array
        input array with moving average applied
    '''
    r, g, b = np.split(np.array(hist), 3)
    return np.concatenate([moving_average(r, w), 
                           moving_average(g, w), 
                           moving_average(b, w)])

Note that in `smooth_histogram()` we're splitting the histogram into its three channels before applying the moving average and re-concatenating them into a single output array. We don't want to blur high-intensity reds and low-intensity blues together, so this extra step is necessary.

Let's apply the smoothing process to our histograms.

In [ ]:
smooth_histograms = [smooth_histogram(h) for h in histograms]

In [ ]:
smooth_similarity = pd.DataFrame(data=[[cosine(h_1, h_2) 
                                        for h_1 in smooth_histograms] 
                                       for h_2 in tqdm(smooth_histograms)],
                                 index=image_ids,
                                 columns=image_ids)

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = smooth_similarity[query_id].sort_values().index.values[1:6]

In [ ]:
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

These results are at least as good as those from the raw histograms, which is reassuring. We'll have to wait until the end of the notebook for a direct comparison though.

# Dimensionality reduction
Another plausible solution to the non-orthogonality of our colour space is to work in some dimensionality reduction. We expect that most neighbours will be correlated with one another (hence the smoothing in the last section), but it's harder to appreciate which neighbourhoods of colour will be more tightly bunched.
Dimensionality reduction techniques like PCA, t-SNE and UMAP are great at finding these correlated features in a high-dimensional space. By keeping the number of target dimensions relatively high (600 from 768) in this case, we restrict the chance of losing any information in the dimensionality reduction process and hopefully retain most of the 'useful' information.

In [ ]:
histograms = np.array(histograms)
histograms.shape

In [ ]:
reduced_histograms = UMAP(n_components=600).fit_transform(histograms)

Our vectors now no longer represent ordered lists of pixel intensities - they're a blur of the 600 most 'useful' aggregated features that could be extracted from the original space. That said, they're still tied to image ids and can be compared to one another in exactly the same way as before. Let's produce our similarity matrix.

In [ ]:
reduced_similarity = pd.DataFrame(data=[[cosine(h_1, h_2) 
                                         for h_1 in reduced_histograms] 
                                        for h_2 in tqdm(reduced_histograms)],
                                  index=image_ids,
                                  columns=image_ids)

and run a test query

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = reduced_similarity[query_id].sort_values().index.values[1:6]

similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

# Direct comparison
Now that we've exhausted approaches to comparing histograms, let's compare the results to one another with the same query image to get a sense of each method's strengths, weaknesses and peculiarities.

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

### naive

In [ ]:
most_similar_ids = naive_similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

### smoothed

In [ ]:
most_similar_ids = smooth_similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

### dimensionality reduction

In [ ]:
most_similar_ids = reduced_similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

### Conclusion
The results from the three approaches are all quite similar, while also being similarly inconsistent.  
On balance, I think the smoothed histogram approach provides the best results. It's good at matching within the greyscale/sepia band of colour space, and it most consistently incorporates colours from outside the band into its results. The dimensionality reduction technique provides surprisingly erratic results - I'm not sure why at this stage.


Whatever the case may be within this histogram-matching experiment, none of the results are really good enough, and there are plenty of other approaches that we could take. We'll explore some alternatives in the next few notebooks.